In [ ]:
%matplotlib widget

from tqdm.notebook import tqdm

import numpy as np

from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt

from lightkurve.search import search_lightcurve

import astropy.units as u

import ipywidgets as widgets
from IPython.display import display

The following "reliable" target list was provided by Kathryn Jones after measuring the Wilson-Bappu relation from OWLS spectra. Targets that appear in this list have "believable" measurements after checking the CaII K line width by eye.

In [ ]:
reliable_wilsonbappu_from_kathyrn = """GJ182
TOI2279
HD189733
WASP-84
K2-100
HAT-P-22
TOI1807
HD78366
K2-182
TOI1416
HD26923
K2-275
TOI1411
Qatar-6
TOI1410
HD3167
HIP116454
TOI2431
TOI1801
TOI2318
HD26913
DSLeo
TOI2018
HD115043
HD72905
GJ490A
K2-184
hd25998
HD131156A
HD75332
HD97658
WASP-69
hd26913
hd82443
TOI2009
K2-291
TOI1683
HD25998
HD131156B
HD129333
GJ140A
HD97334
K2-136
HD88737
TOI1563
K2-141
HD39587
HAT-P-11"""

Make these names closer to the names as they appear on SIMBAD by adding spaces back in:

In [ ]:
def add_spaces(string):
    return string.upper().replace("GJ", "GJ ").replace("HD", "HD ").replace("TOI", "TOI ")

target_list = [add_spaces(name) for name in reliable_wilsonbappu_from_kathyrn.splitlines()]

In [ ]:

reliable_targets = set()
prots = dict()
target_lines = dict()

pbar = tqdm(target_list)

for target in pbar:
    pbar.set_description(target)
    
    s = search_lightcurve(
        target, mission='TESS', author="SPOC"
    ).download_all(
        time_column='bjd', flux_column='sap_flux'
    )

    if s is not None:
        
        lc = s.stitch().remove_outliers().remove_nans()

        x = lc.time.jd-lc.time.jd.mean()
        lc.flux /= np.polyval(np.polyfit(x, lc.flux.value, 3), x)

        periodogram_kwargs = dict(
            minimum_period=1*u.day, maximum_period=40*u.day, 
            nyquist_factor=100, nterms=2, ls_method='fastchi2'
        )
        ls = lc.to_periodogram(**periodogram_kwargs)

        fig = plt.figure(figsize=(10, 5))
        gs = GridSpec(2, 2, figure=fig)
        ax = [
            fig.add_subplot(gs[0, :]),
            fig.add_subplot(gs[1, 0]),
            fig.add_subplot(gs[1, 1]),
        ]
        line0, = ax[0].plot(lc.time.jd - 2457000, lc.flux.value, ',k')

        line1, = ax[1].semilogx(1 / ls.frequency, ls.power, 'k')
        best_period = ls.period[np.nanargmax(ls.power)]
        ax[1].axvline(best_period.to(u.day).value, color='r', ls='--')

        fig.suptitle(f'{target}\n P={best_period}')
        ax[1].set_xscale('log')
        ax[1].set(
            xlabel='Period [d]', ylabel='L-S Power'
        )

        ax[0].plot(lc.time.jd - 2457000, ls.model(lc.time).flux, ',r')

        rot_phase = ((lc.time.jd - 2457000) % best_period.to(u.day).value) / best_period.to(u.day).value
        line2, = ax[2].plot(rot_phase, lc.flux.value, ',k')
        ax[2].plot(rot_phase, ls.model(lc.time).flux, '.r')
        
        ax[2].set(
            xlabel='Rot Phase', ylabel='Flux'
        )
        
        target_lines[target] = [line0, line1, line2]
        
        def plot_function_i(reliable=False, target=target):
            if reliable:
                for line in target_lines[target]:
                    line.set_color('g')
                reliable_targets.add(target)
            else: 
                for line in target_lines[target]:
                    line.set_color('k')
                reliable_targets.remove(target)

        widget = widgets.interactive(plot_function_i, reliable=False, target=target, __line=line)
        controls = widgets.HBox(widget.children[:-1], layout = widgets.Layout(flex_flow='row wrap'))
        output = widget.children[-1]
        display(widgets.VBox([controls, output], layout = widgets.Layout(flex_flow='row wrap')))
        fig.tight_layout()
        fig.canvas.show()
        
        prots[target] = best_period
    pbar.update()

In [ ]:
reliable_targets

In [ ]:
from astropy.table import QTable

table = QTable(
    rows=[[n.replace(" ", ""), np.round(p.value, decimals=1)] 
          for n, p in prots.items() if n in reliable_targets], 
    names=['Target', 'Prot'], units=[None, u.day]
)

In [ ]:
table.pprint(max_lines=50)

In [ ]:
table.write('reliable_prots_for_wilson_bappu.csv')